Import Libraries

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install tensorflow-io

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 28.0/28.0 MB 25.2 MB/s eta 0:00:00


In [3]:
import os
import numpy as np
from matplotlib import pyplot as plt
import tensorflow as tf 
import tensorflow_io as tfio
from tensorflow import keras
from keras import backend as k
import time
from tensorflow.keras.callbacks import EarlyStopping

# 1. Process Audio into Spectogram

a function that returns audio in numeric representation

In [4]:
def load_wav_16k_mono(filename):
    # Load encoded wav file
    file_contents = tf.io.read_file(filename)
    # Decode wav (tensors by channels) 
    wav, sample_rate = tf.audio.decode_wav(file_contents, desired_channels=1)
    # Removes trailing axis
    wav = tf.squeeze(wav, axis=-1)
    sample_rate = tf.cast(sample_rate, dtype=tf.int64)
    # Goes from 44100Hz to 16000hz - amplitude of the audio signal
    #wav = tfio.audio.resample(wav, rate_in=sample_rate, rate_out=16000)
    return wav

Read all audio files and sort

In [5]:
TRAIN = os.path.join('/content','drive','MyDrive','audio-data', 'Train')
TEST = os.path.join('/content','drive','MyDrive','audio-data', 'Test')
#TRAIN = os.path.join('audio-data', 'Train')
#TEST = os.path.join('audio-data', 'Test')
train = tf.data.Dataset.list_files(TRAIN+'/*.wav')
train = sorted(list(train.as_numpy_iterator()))
train = tf.data.Dataset.from_tensor_slices(train)
test = tf.data.Dataset.list_files(TEST+'/*.wav')
test = sorted(list(test.as_numpy_iterator()))
test = tf.data.Dataset.from_tensor_slices(test)

Add Labels

In [6]:
num_classes = 10
iterations = 0
i = 0
train_label = []
while iterations!=len(train):
    iterations +=1
    train_label.append(i)
    i += 1
    if i == num_classes :
        i = 0
train_label=keras.utils.to_categorical(train_label,num_classes)
trainings = tf.data.Dataset.zip((train, tf.data.Dataset.from_tensor_slices(train_label)))
#---------------------------------------------------------------#
iterations = 0
i = 0
test_label=[]
while iterations!=len(test):
    iterations +=1
    test_label.append(i)
    i += 1
    if i == num_classes :
        i = 0
test_label=keras.utils.to_categorical(test_label,num_classes)
testings = tf.data.Dataset.zip((test, tf.data.Dataset.from_tensor_slices(test_label)))

Build Preprocessing Function to get spectogram

In [7]:
def preprocess(file_path, label): 
    wav = load_wav_16k_mono(file_path)
    #wav = wav[:48000]
    #zero_padding = tf.zeros([48000] - tf.shape(wav), dtype=tf.float32)
    #wav = tf.concat([zero_padding, wav],0)
    spectrogram = tf.signal.stft(wav, frame_length=320, frame_step=32)
    spectrogram = tf.abs(spectrogram)
    spectrogram = tf.expand_dims(spectrogram, axis=2)
    return spectrogram, label

Convert all to Spectogram

In [8]:
# train data
x_train = trainings.map(preprocess)
x_train = x_train.cache()
x_train = x_train.shuffle(buffer_size=1000)
x_train = x_train.batch(16) # 16 at a time
x_train = x_train.prefetch(8)
# test data
x_test = testings.map(preprocess)
x_test = x_test.cache()
x_test = x_test.shuffle(buffer_size=1000)
x_test = x_test.batch(16) # 16 at a time
x_test = x_test.prefetch(8)

In [9]:
# test one batch
samples, labels = x_train.as_numpy_iterator().next()
print(samples.shape)
print('\n',labels[0:2],'\n...')

(16, 391, 257, 1)

 [[0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]] 
...


# 2. CNN no Attention

Design the CNN architecture

In [10]:
from keras.models import Sequential
from keras import layers

In [11]:
model=Sequential()
input_shape = (391, 257, 1)
model.add( layers.Conv2D(32,kernel_size=(3,3),activation='relu',input_shape=input_shape) )
model.add( layers.MaxPooling2D(pool_size=(2,2)) )
model.add( layers.Dropout(0.2) )
model.add( layers.Flatten() )
model.add( layers.Dense(32,activation='relu') )
model.add( layers.Dense(num_classes,activation='softmax') )
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 389, 255, 32)      320       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 194, 127, 32)     0         
 )                                                               
                                                                 
 dropout (Dropout)           (None, 194, 127, 32)      0         
                                                                 
 flatten (Flatten)           (None, 788416)            0         
                                                                 
 dense (Dense)               (None, 32)                25229344  
                                                                 
 dense_1 (Dense)             (None, 10)                330       
                                                        

Training

In [12]:
model.compile(optimizer=keras.optimizers.Adam(),
              loss=keras.losses.categorical_crossentropy,
              metrics=['accuracy']
             )

In [13]:
early_stopping = EarlyStopping(monitor='accuracy', patience=3)
tic=time.time()
model.fit(x_train,
          epochs=15,
          verbose=1,
          callbacks=[early_stopping],
          validation_data=x_test
          )
toc=time.time()
training_time=toc-tic

Epoch 1/15
75/75 [==============================] - 66s 668ms/step - loss: 3.2476 - accuracy: 0.2883 - val_loss: 1.6587 - val_accuracy: 0.4967
Epoch 2/15
75/75 [==============================] - 4s 53ms/step - loss: 1.0246 - accuracy: 0.6442 - val_loss: 1.3703 - val_accuracy: 0.6667
Epoch 3/15
75/75 [==============================] - 4s 53ms/step - loss: 0.5822 - accuracy: 0.8383 - val_loss: 1.3431 - val_accuracy: 0.7033
Epoch 4/15
75/75 [==============================] - 4s 53ms/step - loss: 0.3951 - accuracy: 0.8858 - val_loss: 1.4219 - val_accuracy: 0.6933
Epoch 5/15
75/75 [==============================] - 4s 56ms/step - loss: 0.3877 - accuracy: 0.9042 - val_loss: 1.5795 - val_accuracy: 0.7200
Epoch 6/15
75/75 [==============================] - 4s 55ms/step - loss: 0.2621 - accuracy: 0.9375 - val_loss: 1.6066 - val_accuracy: 0.7533
Epoch 7/15
75/75 [==============================] - 4s 57ms/step - loss: 0.1966 - accuracy: 0.9575 - val_loss: 1.5091 - val_accuracy: 0.7800
Epoch 8/15


In [14]:
tic=time.time()
test_loss, test_acc = model.evaluate(x_test)
toc=time.time()
test_time=toc-tic
print("Training Time = {} s".format(np.round(training_time, 1)))
print("Testing Time = {} ms".format(np.round(test_time*1000, 1)))
print('Test Loss = {:.2f} %:'.format(np.round(test_loss, 3)*100))
print('Test Accuracy = {:.2f} %:'.format(np.round(test_acc, 3)*100))

19/19 [==============================] - 0s 8ms/step - loss: 4.5274 - accuracy: 0.6767
Training Time = 128.9 s
Testing Time = 174.5 ms
Test Loss = 452.70 %:
Test Accuracy = 67.70 %:


# 3. CNN with Attention

Design the CNN architecture

In [15]:
inputs = layers.Input(shape=input_shape)
conv = layers.Conv2D(32,kernel_size=(3,3),activation='relu')(inputs)
#Attention
attention = layers.Conv2D(1, (3,3), padding='same', activation='sigmoid')(conv)
attention_mul = layers.Multiply()([conv, attention])
##########
pool = layers.MaxPool2D(pool_size=(2,2))(attention_mul)
drop = layers.Dropout(0.2)(pool)
flatten = layers.Flatten()(drop)
dense = layers.Dense(32,activation='relu')(flatten)
dense2 = layers.Dense(num_classes,activation='softmax')(dense)
modelAtt = keras.Model(inputs=inputs, outputs=dense2)

modelAtt.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 391, 257, 1  0           []                               
                                )]                                                                
                                                                                                  
 conv2d_1 (Conv2D)              (None, 389, 255, 32  320         ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 conv2d_2 (Conv2D)              (None, 389, 255, 1)  289         ['conv2d_1[0][0]']               
                                                                                              

Training

In [16]:
modelAtt.compile(optimizer=keras.optimizers.Adam(),
              loss= keras.losses.CategoricalCrossentropy(),
              metrics=['accuracy']
             )

In [17]:
tic=time.time()
modelAtt.fit(x_train,
          epochs=15,
          verbose=1,
          callbacks=[early_stopping],
          validation_data=x_test
          )
toc=time.time()
training_time=toc-tic

Epoch 1/15
75/75 [==============================] - 9s 89ms/step - loss: 1.6137 - accuracy: 0.5983 - val_loss: 0.8087 - val_accuracy: 0.7100
Epoch 2/15
75/75 [==============================] - 6s 84ms/step - loss: 0.3427 - accuracy: 0.9175 - val_loss: 0.6825 - val_accuracy: 0.8433
Epoch 3/15
75/75 [==============================] - 6s 84ms/step - loss: 0.3376 - accuracy: 0.9217 - val_loss: 0.8374 - val_accuracy: 0.7733
Epoch 4/15
75/75 [==============================] - 6s 85ms/step - loss: 0.2059 - accuracy: 0.9417 - val_loss: 0.9121 - val_accuracy: 0.8200
Epoch 5/15
75/75 [==============================] - 6s 85ms/step - loss: 0.0686 - accuracy: 0.9892 - val_loss: 1.0546 - val_accuracy: 0.8433
Epoch 6/15
75/75 [==============================] - 6s 87ms/step - loss: 0.0282 - accuracy: 0.9983 - val_loss: 1.1979 - val_accuracy: 0.8467
Epoch 7/15
75/75 [==============================] - 6s 86ms/step - loss: 0.0185 - accuracy: 0.9975 - val_loss: 1.2551 - val_accuracy: 0.8567
Epoch 8/15
75

In [18]:
tic=time.time()
test_loss, test_acc = modelAtt.evaluate(x_test)
toc=time.time()
test_time=toc-tic
print("Training Time = {} s".format(np.round(training_time, 1)))
print("Testing Time = {} ms".format(np.round(test_time*1000, 1)))
print('Test Loss = {:.2f} %:'.format(np.round(test_loss, 3)*100))
print('Test Accuracy = {:.2f} %:'.format(np.round(test_acc, 3)*100))

19/19 [==============================] - 0s 14ms/step - loss: 1.8363 - accuracy: 0.8533
Training Time = 83.3 s
Testing Time = 310.9 ms
Test Loss = 183.60 %:
Test Accuracy = 85.30 %:
